# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


### Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('/datasets/data.csv')

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


При просмотре статистики по датасету можно увидитить множество ошибок в нескольких столбца children min количество детей -1, days_employed отрицательные значения и слишком завышенное количество отработанных дней, если смотреть на количество данных в дата сете то в нескольких столбщах их будет не хватать  days_employed	и total_income.

### Шаг 2.1 Заполнение пропусков

Можно заметить, что значения в 'days_employed' и 'total_income' отсутствуют.

Заменим их средними значениями для того, чтобы они не искажали выходные данные но остальные их параметры можно использовать для анализа.

In [6]:
data['days_employed']=data['days_employed'].fillna(data['days_employed'].mean())

In [7]:
data['total_income']=data['total_income'].fillna(data['total_income'].mean())

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.2 Проверка данных на аномалии и исправления.

Проверим на уникальность столбец образования и найдём все аномальные данные.

In [9]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Получив столбец можно заметить что все его значения отличаются разными регистрами приведём всё к одному регистру.

In [10]:
data['education'].str.lower().unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Заметим что не никаких больше аномалий можно заменить все аномальные данные на нормальны просто преобразовав их в нижний регистр.

In [11]:
data['education']=data['education'].str.lower()

Сделаем все те же самые операции для семейного статуса.

In [12]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Нет никаких аномалий, проврим столбец с типом занятости.

In [13]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Аномалий тоже нет, проврим что со столбцом рабочих дней.

In [14]:
data[data['days_employed']<0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Заметим что многие значения отрицательны, приведём их к абсолютному значению.

In [15]:
data['days_employed']=abs(data['days_employed'])

Заменю данные на основе возраста, на максимальный стаж для данного возраста.

'(data['dob_years']-14)*365' подсчёт максимального возраста стажа для человека 

"(data[data['days_employed']>(data['dob_years']-14)*365]['dob_years']-14)*365" подсчёт максимального стажа для замены

In [16]:
data.loc[data['days_employed']>(data['dob_years']-14)*365,'days_employed']=(data[data['days_employed']>(data['dob_years']-14)*365]['dob_years']-14)*365

Проверим на данные выходящие из ряда.

In [17]:
data[data['days_employed']>(data['dob_years']-14)*365]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Все данные находятся в нормальных границах.

In [18]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14235.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,19345.000000,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Шаг 2.3. Изменение типов данных.

Нужно заменить все столбцы в которых предполагаются точные цифры и заменить все типы данных на более корректные для уменьшения обрабатываемого объёма данных.

Такие как количество отработанных дней и зарплата.

In [19]:
data['days_employed']=data['days_employed'].astype('int64')

In [20]:
data['total_income']=data['total_income'].astype('int64')

In [21]:
data['children']=data['children'].astype('int8')

In [22]:
data['dob_years']=data['dob_years'].astype('int8')

In [23]:
data['family_status_id']=data['family_status_id'].astype('int8')

In [24]:
data['debt']=data['debt'].astype('int8')

In [25]:
data['education_id']=data['education_id'].astype('int8')

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int8  
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int8  
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int8  
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int8  
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int8  
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(2), int8(5), object(5)
memory usage: 1.3+ MB


Заменив все данные на более подходящие у датасет размер датасета уменьшился на .7мб.

### Шаг 2.4. Удаление дубликатов.

Используем самый явный способ для выявления дубликатов и посчитаем их кол-во:

In [27]:
data.duplicated().sum()

71

Удалим все дубликаты и переопределим индексы.

In [28]:
data=data.drop_duplicates().reset_index(drop=True)

In [29]:
data.duplicated().sum()

0

Дубликаты могут появляться из-за нескольких вещей, во-первых из-за ошибки человке при внесении данных, во-вторых ошибка на стороне компьютера, отправленно несколько запросов на внесение данных в базу данных или ошибки при отправке данных.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Узнаем уникальные значения для столбца образования, чтобы потом по ним сформировать Series.

In [30]:
unique_educ=data['education'].unique()

In [31]:
unique_educ

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [32]:
unique_educ_id=data['education_id'].unique()

In [33]:
unique_educ_id

array([0, 1, 2, 3, 4], dtype=int8)

Создаём Series с индексами имеющими соответствующее значение для образования.

In [34]:
data_educ=pd.Series(data=unique_educ, index=unique_educ_id)

In [35]:
data_educ

0                 высшее
1                среднее
2    неоконченное высшее
3              начальное
4         ученая степень
dtype: object

Повторяем всё тоже самое для семейного положения.

In [36]:
unique_fs=data['family_status'].unique()

In [37]:
unique_fs

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [38]:
unique_fs_id=data['family_status_id'].unique()

In [39]:
unique_fs_id

array([0, 1, 2, 3, 4], dtype=int8)

In [40]:
data_fs=pd.Series(data=unique_fs, index=unique_fs_id)

In [41]:
data_fs

0          женат / замужем
1         гражданский брак
2           вдовец / вдова
3                в разводе
4    Не женат / не замужем
dtype: object

Удаляем колонки для которых создали отдельные Seried данных.

In [42]:
data=data.drop(columns=['family_status','education'])

In [43]:
data

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,14235,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем
21450,0,19345,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость
21452,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Шаг 2.6. Категоризация дохода.

Создадим функцию для категоризации данных по уровню их дохода.

In [44]:
def category(data):
    if data['total_income']<30000:
         return 'E'
    elif data['total_income']<=50000:
         return 'D'
    elif data['total_income']<=200000:
         return 'C'
    elif data['total_income']<=1000000:
         return 'B'
    elif data['total_income']>1000000:
         return 'A'
    else:
        print('Некоректное значение таблицы data')
        return 0

Применим эти данных к данным и поместим их в отдельный столбец.

In [45]:
data['total_income_category']=data.apply(category, axis=1)

In [46]:
data

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14235,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем,B
21450,0,19345,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21451,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость,C
21452,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B


### Шаг 2.7. Категоризация целей кредита.

Узнаем все уникальные цели для кредита и выделим ключевые слова.

In [47]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функциюиспользующую эти ключевые слова и их значения при этом в перебор помещаю коммерческую недвижимость так как в поиске для неё не используется слово недвижимость, и тем самым исключаем нкоммерческую недвижимость при поиске в жилой недвижимости. 

In [48]:
def cat_purpose(data):
    hom=[['жил','недвижимост'],['жилая недвижимость']]
    com_hom=[['сдач','коммерческ'],['коммерческая недвижимость']]
    auto=[['авто'],['покупка автомобиля']]
    educ=[['образован', 'высше'],['получение образования']]
    com=[['свадь'],['организяция свадьбы']]
    try:
        for categ in [com_hom, hom, auto, educ, com]:
            for arr in categ[0]:
                if arr in data['purpose']:
                    return categ[1][0]
    except:
        print('Ошибка в данных.')
        exit()
    return 0

Применяем полученную функцию к данным data.

In [49]:
data['purpose_category']=data.apply(cat_purpose, axis=1)

In [50]:
data

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,жилая недвижимость
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,покупка автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,жилая недвижимость
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14235,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,организяция свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем,B,жилая недвижимость
21450,0,19345,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,покупка автомобиля
21451,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость,C,жилая недвижимость
21452,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,покупка автомобиля


Проверка на нули

In [51]:
data[data['purpose_category']==0]

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category


### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Подсчёт количество отданных кредитов без задолженностей по кредиту для людей с детьми.

In [52]:
data[(data['children']>=1) & (data['debt']==0)]['debt'].count()

6639

Подсчёт количество отданных кредитов с задолженностей по кредиту для людей с детьми.

In [53]:
data[(data['children']>=1) & (data['debt']==1)]['debt'].count()

677

Процентное значение для людей с задолженность по возврату кредитов с детьми.

In [54]:
data[(data['children']>=1) & (data['debt']==1)]['debt'].count()/(data[(data['children']>=1) & (data['debt']==0)]['debt'].count()+data[(data['children']>=1) & (data['debt']==1)]['debt'].count())*100

9.253690541279386

Подсчёт количество отданных кредитов без задолженностей по кредиту для людей бездетьми.

In [55]:
data[(data['children']==0) & (data['debt']==0)]['debt'].count()

13028

Подсчёт количество отданных кредитов с задолженностей по кредиту для людей без детьми.

In [56]:
data[(data['children']==0) & (data['debt']==1)]['debt'].count()

1063

Процентное значение для людей с задолженность по возврату кредитов без детьми.

In [57]:
data[(data['children']==0) & (data['debt']==1)]['debt'].count()/(data[(data['children']==0) & (data['debt']==0)]['debt'].count()+data[(data['children']==0) & (data['debt']==1)]['debt'].count())*100

7.543822297920659

##### Вывод 1:

Разница между заёмщиками с детьм и без детей, сильно разница, а точнее на 1,7% эта разница существенна для банков и они будут отдавать большее предпочтение к людям без детей.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [58]:
data_deb=pd.DataFrame()

In [59]:
data_deb['family_status_id']=data[(data['debt']==0)].groupby('family_status_id')['family_status_id'].agg('count')

In [60]:
data_deb['family_status_id_deb']=data[(data['debt']==1)].groupby('family_status_id')['family_status_id'].agg('count')

In [61]:
data_deb['proz']=data_deb['family_status_id_deb']/(data_deb['family_status_id_deb']+data_deb['family_status_id'])

In [62]:
data_deb

,family_status_id,family_status_id_deb,proz
family_status_id,,,
0,11408,931,0.075452
1,3763,388,0.093471
2,896,63,0.065693
3,1110,85,0.071130
4,2536,274,0.097509


##### Вывод 2:

Процентный показатель количества затруднений в возврщении долга, сильно завышен для "Не женат / не замужем" и занижен для "вдовец / вдова". Из отношений можно сделать вывод, что люди состоявшие или состоящие в браке более надёжные чем люди состоящие в гражданском браке или вообще не женат / не замужем

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [63]:
data_deb=pd.DataFrame()

In [64]:
data_deb['total_income_category']=data[(data['debt']==0)].groupby('total_income_category')['total_income_category'].agg('count')

In [65]:
data_deb['total_income_category_deb']=data[(data['debt']==1)].groupby('total_income_category')['total_income_category'].agg('count')

In [66]:
data_deb['proz']=data_deb['total_income_category_deb']/(data_deb['total_income_category_deb']+data_deb['total_income_category'])

In [67]:
data_deb

,total_income_category,total_income_category_deb,proz
total_income_category,,,
A,23,2,0.080000
B,4685,356,0.070621
C,14656,1360,0.084915
D,329,21,0.060000
E,20,2,0.090909


##### Вывод 3:

Значения для категории D, E и A являются некорректными так как слишком мальнькая выборка, их можно убрать из выборки. Среднее значение для B и C является 7,7767995, это значение не сильно отличается от общего среднего значения, разница между B и C составляет ~1.5%, что является существенным для выборки, и достаток выше среднего сильно понижает риски не платёжеспособности.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [68]:
data_deb=pd.DataFrame()

In [69]:
data_deb['purpose_category']=data[(data['debt']==0)].groupby('purpose_category')['purpose_category'].agg('count')

In [70]:
data_deb['purpose_category_deb']=data[(data['debt']==1)].groupby('purpose_category')['purpose_category'].agg('count')

In [71]:
data_deb['proz']=data_deb['purpose_category_deb']/(data_deb['purpose_category']+data_deb['purpose_category_deb'])

In [72]:
data_deb

,purpose_category,purpose_category_deb,proz
purpose_category,,,
жилая недвижимость,8218,631,0.071307
коммерческая недвижимость,1811,151,0.076962
организяция свадьбы,2138,186,0.080034
покупка автомобиля,3903,403,0.093590
получение образования,3643,370,0.092200


##### Вывод 4:

Менее всего подвержены риску заёмщики на недвижимость как жилую так и коммерческу, так как они увеличивают благосостояние жизни, но образование и покупка машины, более подвержены риску из-за невозможности найти высокооплачиваемой работы заёмщиком чтобы отдать кредит, или возможностью повреждения машины, что требует дополнительных затрат на машину. 

## Общий вывод:

Анализируя заёмщиков на предмет неплатёжеспособности, можно заметить факторы которые увеличиют возможность невозврата долга, такие как 'не женатый/не замужняя' этот фактор больше всего влияет на невозварат долга, далее идёт присутствие только 1 ребёнка и если цель кредита машина. Факторы которые уменьшают шанс на невозварат догла это 'женатый/замужний', если берут кредит на квартиру/бизнес и отсутствие детей. Невозможно правильно судить про достатку людей из-за крайне малом объёме данных.